In [4]:
## do tokenize
## return tokenized list

class Tokenizer:

    def __init__(self, n_gram, stop_en=None, stop_th=None, keyword=None):

        import re
        import deepcut
        import os
        from nltk.tokenize import TreebankWordTokenizer
        from nltk.stem.snowball import EnglishStemmer

        self.test_text = 'ตัวอย่างความต้องการใช้ตัวอย่างความต้องการลีนุ๊กซ์การใช้ยากลำบาก'
        self.pattern_thai_char = re.compile(u'[\u0e00-\u0e7f]')
        self.pattern_new_sentence = re.compile('\.[0-9]+(\)|\.) ')
        self.pattern_th_out = re.compile(u'[\u0e00-\u0e7f][^\u0e00-\u0e7f]')
        self.pattern_th_in = re.compile(u'[^\u0e00-\u0e7f][\u0e00-\u0e7f]')
        self.pattern_num_bullet = re.compile('^[0-9]+(\)|\.)*$')
        self.pattern_end_token = re.compile('^[a-zA-Z]+$')
        self.pattern_number = re.compile('\+*[0-9]+')
        self.pattern_phone_number = re.compile('[0-9]+-[0-9]+-[0-9]+')
        self.pattern_email = re.compile('[a-zA-Z._\-0-9]+@[a-zA-Z._\-0-9]+')
        self.pattern_url = re.compile('(https://|www.)[a-zA-Z0-9]+.[a-z]+[^\s]*')
        self.pattern_sentence_collide = re.compile('[a-z][A-Z]]')
        self.pattern_thai_name = re.compile(u'\u0e04\u0e38\u0e13\s*[\u0e00-\u0e7f]+\s+')
        self.charset = {}
        with open(os.path.join(os.getcwd(), '..', 'dict', 'charset'), 'rt') as charfile:
            for item in charfile.read().split('\n'):
                if len(item) < 4:
                    self.charset[item] = ord(item)
                else:
                    self.charset[chr(int(item, 16))] = int(item, 16)
        self.eng_tokenizer = TreebankWordTokenizer()
        self.stemming = EnglishStemmer()
        self.n_gram = n_gram
        self.dp = deepcut
        if stop_en:
            with open(os.path.join(os.getcwd(), '..', 'dict', stop_en), 'rt', encoding='utf-8') as stop_file:
                self.stop_en = set([item for item in stop_file.read().split('\n')])
        else:
            self.stop_en = set([])
        if stop_th:
            with open(os.path.join(os.getcwd(), '..', 'dict', stop_th), 'rt', encoding='utf-8') as stop_file:
                self.stop_th = set([item for item in stop_file.read().split('\n')])
        else:
            self.stop_th = set([])
        if keyword:
            with open(os.path.join(os.getcwd(), '..', 'dict', keyword), 'rt', encoding='utf-8') as keyword_file:
                self.keyword = set([item for item in keyword_file.read().split('\n')])
        else:
            self.keyword = set([])

    def tokenizer(self, text=None):

        def n_gram_compile(tokens, n):

            tokens = tokens[:]
            n_tokens = []
            if n <= 1:
                return tokens
            for j, token in enumerate(tokens[:-(n - 1)]):
                new_token = ''
                for word in tokens[j:j + n]:
                    if self.pattern_thai_char.search(word) and len(word) > 1:
                        new_token += word
                    else:
                        new_token = ''
                        break
                if new_token:
                    n_tokens.extend([new_token])
            return n_tokens

        def n_grams_compile(tokens, n):

            if n < 2:
                return tokens
            n_tokens = []
            for j in range(2, n + 1):
                n_tokens.extend(n_gram_compile(tokens, j))
            n_tokens = tokens + n_tokens
            return n_tokens

        def validate_char(val_text):
            val_text = val_text.replace('&amp;', ' ')
            val_text = val_text.replace('&nbsp;', ' ')
            ret_text = ''
            for cha in val_text:
                try:
                    self.charset[cha]
                except KeyError:
                    ret_text += ' '
                else:
                    ret_text += cha
            while ret_text.find('  ') != -1:
                ret_text = ret_text.replace('  ', ' ')
            return ret_text

        def split_th_en(splt_text):
            insert_pos = []
            splt_text = splt_text[:]
            for pos, item in enumerate(splt_text[:-2]):
                if self.pattern_th_in.search(splt_text[pos:pos+2]) or self.pattern_th_out.search(splt_text[pos:pos+2]):
                    insert_pos.append(pos + 1)
            for pos in reversed(insert_pos):
                splt_text = splt_text[:pos] + ' ' + splt_text[pos:]
            return splt_text

        def remove_thai_stop(th_text):
            stop_pos = [[0, 0]]
            ## TH : do longest matching
            for j in range(len(th_text)-1):
                for k in range(j+1, len(th_text)):
                    if th_text[j:k] in self.stop_th:
                        # found keyword +++ instead of returning string - return positions that is
                        # i to j
                        if j <= stop_pos[-1][1]:
                            stop_pos[-1] = [stop_pos[-1][0], k]
                        else:
                            stop_pos.append([j, k])
                        break
            newstr = ''
            if len(stop_pos) == 1:
                newstr = th_text
            else:
                for j in range(len(stop_pos)-1):
                    newstr += th_text[stop_pos[j][1]:stop_pos[j+1][0]] + ' '
            return newstr

        if text == '-test':
            text = self.test_text

        text = text.replace(u'\u0e46', ' ')
        text = self.pattern_email.sub(' ', text)
        text = self.pattern_url.sub(' ', text)
        text = self.pattern_phone_number.sub(' ', text)
        text = self.pattern_thai_name.sub(' ', text)
        text = split_th_en(text)
        text = self.pattern_new_sentence.sub(' . ', text)
        text = text.replace('.', ' . ')
        text = validate_char(text)
        text = remove_thai_stop(text)
        text_split = text.split(' ')
        text_split = [item for item in text_split[:] if item not in self.stop_en
                      and not self.pattern_num_bullet.search(item)]
        text_split = [self.stemming.stem(item) if self.pattern_end_token.search(item) and
                      item not in self.keyword else item for item in text_split[:]]

        first_pass = []
        for i, item in enumerate(text_split):
            if self.pattern_sentence_collide.search(item) and item not in self.keyword:
                c_text = self.pattern_sentence_collide.search(item)
                first_pass.extend([c_text.string[:c_text.span()[0]+1], c_text.string[c_text.span()[1]-1:]])
            else:
                first_pass.append(item)
        second_pass = []
        for i, chunk in enumerate(first_pass):
            if self.pattern_thai_char.search(chunk) and len(chunk) > 1:
                new_chunk = self.dp.tokenize(chunk)
                second_pass.extend(new_chunk)
            else:
                second_pass.append(chunk.lower())

        second_pass = n_grams_compile(second_pass, self.n_gram)

        return set(second_pass)

In [5]:
## Construct Dataframe

import pandas as pd

class DataController():
    dataMatrix = pd.DataFrame(columns=["title","desc","tag"])
    
    ## init will create dataMatrix
    def __init__(self, pathToFile):
        import os
        import json
        count = 0
        
        with open(pathToFile, 'r', encoding='utf-8') as fin:
            for line in fin:
                ## for each line, add into dataMatrix, using ["title", "desc", "tag"] structure
                line_dict = json.loads(line, encoding='utf-8')
                self.dataMatrix = self.dataMatrix.append(line_dict, ignore_index=True)
                #count+=1
                #if(count==100): break
    
    def getTrainingSet(self, label_class):
        ## classSet is set of data that has tag = label_class
        targetSet = self.dataMatrix[self.dataMatrix['tag']==label_class]
        restSet = self.dataMatrix[self.dataMatrix['tag']!=label_class]

        if(targetSet.shape[0] < restSet.shape[0]):
            # target has less population than the rest
            trainingSet = pd.concat([targetSet, restSet.sample(n=targetSet.shape[0])])
        else:
            # target has more population than the rest
            trainingSet = pd.concat([targetSet.sample(n=restSet.shape[0]), restSet])
        # shuffle data using sample fraction = 1
        trainingSet = trainingSet.sample(frac=1)
        return trainingSet

In [6]:
## Create data
import os

file_name = "block1234.json"
file_path = os.getcwd()+"/../data/"+file_name

data = DataController(file_path)

In [7]:
## Create training data
trainingData = data.getTrainingSet("0")

training_Desc = trainingData['desc'] 
training_Title = trainingData['title']
training_Label = trainingData['tag']

In [8]:
data.dataMatrix[(data.dataMatrix['tag'] == "0")].shape

(1455, 3)

In [9]:
## create tokenizer
tkn1 = Tokenizer(1, 'en_stop_word.txt', 'th_stop_word.txt')
tkn2 = Tokenizer(2, 'en_stop_word.txt', 'th_stop_word.txt')
tkn3 = Tokenizer(3, 'en_stop_word.txt', 'th_stop_word.txt')
tkn4 = Tokenizer(4, 'en_stop_word.txt', 'th_stop_word.txt')

## open vocab file
#import os
#with open(os.path.abspath(os.path.join(os.getcwd(), '..', 'dict', 'desc_newdict_90p.txt'))  , 'rt', encoding='utf-8') as f_tv:
#    desc_vocab = f_tv.read().split('\n')
#with open(os.path.abspath(os.path.join(os.getcwd(), '..', 'dict', 'title_newdict_90p.txt'))  , 'rt', encoding='utf-8') as f_tv:
#    title_vocab = f_tv.read().split('\n')

## create tfidf term-doc matrix
from sklearn.feature_extraction.text import TfidfVectorizer

desc_vectorizer = TfidfVectorizer(tokenizer=tkn2.tokenizer)
desc_vec = desc_vectorizer.fit_transform(training_Title)

title_vectorizer = TfidfVectorizer(tokenizer=tkn4.tokenizer)
title_vec = title_vectorizer.fit_transform(training_Desc)

## stack title onto desc
from scipy.sparse import hstack
data_vec = hstack([title_vec, desc_vec])

## create label_vec
label_vec = training_Label

Using TensorFlow backend.


In [204]:
## Train using Bernoulli NaiveBayes 
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB

## see crossvalidation score
bclf = BernoulliNB()
scores = cross_val_score(bclf, data_vec, label_vec, cv=3, scoring='f1_macro')
print(scores)

## see train_test_split
from sklearn.model_selection import train_test_split
desc_train, desc_test, label_train, label_test = train_test_split(data_vec, label_vec, test_size=0.3)
bclf = bclf.fit(desc_train, label_train)
label_predict = bclf.predict(desc_test)


from sklearn.metrics import classification_report
print(classification_report(label_test, label_predict))
in_bclf = BernoulliNB()
in_bclf = in_bclf.fit(data_vec, label_vec)
label_predict = bclf.predict(data_vec)

from sklearn.metrics import classification_report
print(classification_report(label_vec, label_predict))

[0.77631149 0.77475156 0.76089342]
             precision    recall  f1-score   support

          0       0.85      0.79      0.82       106
          1       0.79      0.85      0.82        99

avg / total       0.82      0.82      0.82       205

             precision    recall  f1-score   support

          0       0.94      0.85      0.89       341
          1       0.86      0.95      0.90       341

avg / total       0.90      0.90      0.90       682



In [205]:
## Train using Multinomial NaiveBayes 
def train_MNB(alpha):
    from sklearn.model_selection import cross_val_score
    from sklearn.naive_bayes import MultinomialNB 

    ## see crossvalidation score
    mclf = MultinomialNB(alpha=alpha)
    scores = cross_val_score(mclf, data_vec, label_vec, cv=3, scoring='f1_macro')
    return sum(scores)/3

    ## see train_test_split
    #from sklearn.model_selection import train_test_split
    #desc_train, desc_test, label_train, label_test = train_test_split(data_vec, label_vec, test_size=0.3)
    #mclf = mclf.fit(desc_train, label_train)
    #label_predict = mclf.predict(desc_test)


    #from sklearn.metrics import classification_report
    #print(classification_report(label_test, label_predict))
    #in_mclf = MultinomialNB(alpha=alpha)
    #in_mclf = in_mclf.fit(data_vec, label_vec)
    #label_predict = mclf.predict(data_vec)

    #from sklearn.metrics import classification_report
    #print(classification_report(label_vec, label_predict))

In [206]:
from scipy.optimize import newton
import random
roots = [newton(train_MNB, x0) for x0 in [random.random() + 0.3 for i in range(10)]]
for item in roots:
    print(train_MNB(item), item)

/usr/share/anaconda3/lib/python3.6/site-packages/scipy/optimize/zeros.py:195: RuntimeWarning: Tolerance of 0.0001934855588023865 reached
  warnings.warn(msg, RuntimeWarning)
/usr/share/anaconda3/lib/python3.6/site-packages/scipy/optimize/zeros.py:195: RuntimeWarning: Tolerance of 0.00016094804132793517 reached
  warnings.warn(msg, RuntimeWarning)
/usr/share/anaconda3/lib/python3.6/site-packages/scipy/optimize/zeros.py:195: RuntimeWarning: Tolerance of 0.00017669749049931127 reached
  warnings.warn(msg, RuntimeWarning)
/usr/share/anaconda3/lib/python3.6/site-packages/scipy/optimize/zeros.py:195: RuntimeWarning: Tolerance of 0.00016553578078248865 reached
  warnings.warn(msg, RuntimeWarning)
/usr/share/anaconda3/lib/python3.6/site-packages/scipy/optimize/zeros.py:195: RuntimeWarning: Tolerance of 0.00018166835026145467 reached
  warnings.warn(msg, RuntimeWarning)
/usr/share/anaconda3/lib/python3.6/site-packages/scipy/optimize/zeros.py:195: RuntimeWarning: Tolerance of 0.00022757640132797

0.8453096457733748 0.9349523308031198
0.8438149905464515 0.6095608873005036
0.8438149905464515 0.7670632537381816
0.8438149905464515 0.6554405757158426
0.8438149905464515 0.8167743367902573
0.8408867404466127 1.275877801480601
0.8408867404466127 1.2544635588703295
0.8438149905464515 0.7461034043232557
0.8438202539079657 0.9842068285229519
0.8452233052756193 0.4267374609873982


In [207]:
alphas = [item / 10 for item in range(10,21,1)]
for item in alphas:
    print(train_MNB(item), item)

0.8423775470079425 1.0
0.8408867404466127 1.1
0.8408867404466127 1.2
0.8408867404466127 1.3
0.8408867404466127 1.4
0.8393920852196896 1.5
0.840913932213803 1.6
0.8394156698496268 1.7
0.8394156698496268 1.8
0.8394156698496268 1.9
0.8394156698496268 2.0


In [208]:
label_vec.shape

(682,)

In [209]:
desc_train, desc_test, label_train, label_test = train_test_split(data_vec, label_vec, test_size=0.3)
mclf = mclf.fit(desc_train, label_train)
label_predict_prob = mclf.predict_proba(desc_test)
label_predict = mclf.predict(desc_test)
label_predict_prob[label_predict != label_test]

array([[3.77111972e-01, 6.22888028e-01],
       [2.12489120e-01, 7.87510880e-01],
       [2.11919943e-01, 7.88080057e-01],
       [5.33555688e-01, 4.66444312e-01],
       [8.03297635e-02, 9.19670236e-01],
       [3.64855723e-01, 6.35144277e-01],
       [1.25997374e-01, 8.74002626e-01],
       [7.88665460e-01, 2.11334540e-01],
       [9.49491211e-01, 5.05087885e-02],
       [1.98547732e-02, 9.80145227e-01],
       [4.23958298e-03, 9.95760417e-01],
       [5.62176866e-01, 4.37823134e-01],
       [3.40298525e-01, 6.59701475e-01],
       [2.54083625e-01, 7.45916375e-01],
       [9.39635555e-01, 6.03644451e-02],
       [7.05110852e-04, 9.99294889e-01],
       [4.99019865e-01, 5.00980135e-01],
       [1.89964852e-02, 9.81003515e-01],
       [8.95617379e-01, 1.04382621e-01],
       [7.16911508e-01, 2.83088492e-01],
       [8.46527875e-01, 1.53472125e-01],
       [7.42176098e-01, 2.57823902e-01],
       [2.79384566e-01, 7.20615434e-01]])

In [240]:
frame = pd.DataFrame(index=label_test[label_predict != label_test].index.values)
frame['Label'] = list(label_test[label_predict != label_test])
frame['prob_0'] = list(label_predict_prob[label_predict != label_test][:,0])
frame['prob_1'] = list(label_predict_prob[label_predict != label_test][:,1])
frame['title'] = [training_Title[item] for item in frame.index.values]
frame['desc'] = [training_Desc[item] for item in frame.index.values]
frame
frame.to_csv('classification_error.csv', encoding='utf-8', sep='\t')

In [237]:
print(training_Title[276,323])
print(training_Desc[276])

KeyError: (276, 323)

In [12]:
## Train using Multinomial NaiveBayes 
def train_MNB(alpha):
    from sklearn.model_selection import cross_val_score
    from sklearn.naive_bayes import MultinomialNB 

    ## see crossvalidation score
    mclf = MultinomialNB(alpha=alpha)
    scores = cross_val_score(mclf, data_vec, label_vec, cv=3, scoring='f1_macro')

    ## see train_test_split
    from sklearn.model_selection import train_test_split
    desc_train, desc_test, label_train, label_test = train_test_split(data_vec, label_vec, test_size=0.3)
    mclf = mclf.fit(desc_train, label_train)
    label_predict = mclf.predict(desc_test)


    from sklearn.metrics import classification_report
    print(classification_report(label_test, label_predict))
    in_mclf = MultinomialNB(alpha=alpha)
    in_mclf = in_mclf.fit(data_vec, label_vec)
    label_predict = mclf.predict(data_vec)

    from sklearn.metrics import classification_report
    print(classification_report(label_vec, label_predict))

In [13]:
train_MNB(alpha=1)

             precision    recall  f1-score   support

          0       0.94      0.79      0.86       112
          1       0.79      0.94      0.86        93

avg / total       0.87      0.86      0.86       205

             precision    recall  f1-score   support

          0       0.97      0.87      0.91       341
          1       0.88      0.97      0.92       341

avg / total       0.92      0.92      0.92       682

